# GPU Validation

In [ ]:
import tensorflow as tf

gpus = tf.config.list_physical_devices('GPU')

if gpus:
    print(f"GPU(s) available: {len(gpus)}")
    for gpu in gpus:
        print(f"  - {gpu.name}")
    if tf.test.is_built_with_cuda():
        print("TensorFlow is built with CUDA support.")
    else:
        print("TensorFlow is NOT built with CUDA support.")
else:
    print("No GPU devices found.")

GPU(s) available: 1
  - /physical_device:GPU:0
TensorFlow is built with CUDA support.


Instalar Cupy

In [ ]:
!pip install cupy-cuda12x --quiet

In [ ]:
import time
import cupy as cp
import numpy as np

# Lectura CSV

In [ ]:
import pandas as pd
df_spy = pd.read_csv('spy_options_call_output_all.csv')

In [ ]:
# Ordenar cronológicamente el dataset
df_spy = df_spy.sort_values(by=['quote_date', 'expire_date'], ascending=True).reset_index(drop=True)

In [ ]:
# Verificar si esta ordenado correctamente
print(df_spy['quote_date'].head())
print(df_spy['quote_date'].tail())

0    2020-01-02
1    2020-01-02
2    2020-01-02
3    2020-01-02
4    2020-01-02
Name: quote_date, dtype: object
154836    2023-12-29
154837    2023-12-29
154838    2023-12-29
154839    2023-12-29
154840    2023-12-29
Name: quote_date, dtype: object


In [ ]:
df_spy.head()

,quote_date,expire_date,dte,underlying_last,strike,strike_distance,strike_distance_pct,c_last,c_bid,c_ask,...,relative_spread,dte_year,c_volume_log,c_volume_log_scaled,volume_zero,moneyness,iv_pct_rank,c_iv_min,iv_skew,spread_mid_price
0,2020-01-02,2020-01-03,1.0,324.87,331.5,6.6,0.020,0.05,0.0,0.02,...,2.0,0.003968,2.708050,0.396255,0,0.980000,0.615385,0.07086,0.03798,2.0
1,2020-01-02,2020-01-03,1.0,324.87,332.0,7.1,0.022,0.01,0.0,0.02,...,2.0,0.003968,1.945910,0.284735,0,0.978524,0.628205,0.07086,0.04462,2.0
2,2020-01-02,2020-01-03,1.0,324.87,332.5,7.6,0.023,0.02,0.0,0.02,...,2.0,0.003968,3.688879,0.539775,0,0.977053,0.692308,0.07086,0.05172,2.0
3,2020-01-02,2020-01-03,1.0,324.87,336.0,11.1,0.034,0.03,0.0,0.02,...,2.0,0.003968,4.304065,0.629792,0,0.966875,0.846154,0.07086,0.10017,2.0
4,2020-01-02,2020-01-03,1.0,324.87,340.0,15.1,0.047,0.01,0.0,0.02,...,2.0,0.003968,0.693147,0.101425,0,0.955500,0.910256,0.07086,0.15350,2.0


In [ ]:
df_spy.shape

(154841, 31)

# Monte Carlo Simulation (GPU) - LSM

In [ ]:
import cupy as cp

def american_call_longstaff_schwartz_gpu(
    S0, K, r, sigma, T, paths=2000, steps=100
):
    dt = T / steps
    discount = cp.exp(-r * dt)

    # Simular paths en GPU
    S = cp.zeros((paths, steps+1))
    S[:, 0] = S0

    for t in range(1, steps+1):
        Z = cp.random.normal(size=paths)
        S[:, t] = S[:, t-1] * cp.exp((r - 0.5*sigma**2)*dt + sigma*cp.sqrt(dt)*Z)

    # Payoff al vencimiento
    payoff = cp.maximum(S[:, -1] - K, 0)

    # Backward induction (LSMC)
    for t in range(steps-1, 0, -1):
        itm = S[:, t] > K  # in-the-money

        if cp.sum(itm) == 0:
            payoff = discount * payoff
            continue

        X = S[itm, t]
        Y = payoff[itm] * discount

        # Ajuste polinómico (CuPy GPU)
        coeff = cp.polyfit(X, Y, 2)
        continuation = cp.polyval(coeff, X)

        exercise = X - K
        payoff[itm] = cp.where(exercise > continuation, exercise, discount * payoff[itm])

    # Pasar el resultado a CPU (float)
    return float(cp.mean(payoff).get())

In [ ]:
start_time = time.time()

df_spy["mc_price"] = df_spy.apply(
    lambda row: american_call_longstaff_schwartz_gpu(
        S0=row["underlying_last"],
        K=row["strike"],
        r=0.05,
        sigma=row["c_iv"],
        T=row["dte_year"]
    ), axis=1
)

end_time = time.time()
print(f"LSM - Monte Carlo Simulation time (GPU): {end_time - start_time} seconds")

Streaming output truncated to the last 5000 lines.
/usr/local/lib/python3.12/dist-packages/pandas/core/apply.py:1081: RankWarning: Polyfit may be poorly conditioned
  results[i] = self.func(v, *self.args, **self.kwargs)
/usr/local/lib/python3.12/dist-packages/pandas/core/apply.py:1081: RankWarning: Polyfit may be poorly conditioned
  results[i] = self.func(v, *self.args, **self.kwargs)
/usr/local/lib/python3.12/dist-packages/pandas/core/apply.py:1081: RankWarning: Polyfit may be poorly conditioned
  results[i] = self.func(v, *self.args, **self.kwargs)
/usr/local/lib/python3.12/dist-packages/pandas/core/apply.py:1081: RankWarning: Polyfit may be poorly conditioned
  results[i] = self.func(v, *self.args, **self.kwargs)
/usr/local/lib/python3.12/dist-packages/pandas/core/apply.py:1081: RankWarning: Polyfit may be poorly conditioned
  results[i] = self.func(v, *self.args, **self.kwargs)
/usr/local/lib/python3.12/dist-packages/pandas/core/apply.py:1081: RankWarning: Polyfit may be poorly co

In [ ]:
print(f"LSM - Monte Carlo Simulation time (GPU): {(end_time - start_time)/60} minutes")

In [ ]:
df_spy[df_spy['mc_price'] > 0][['mc_price', 'c_mid_price', 'c_last']].head(10)

##Generar CSV

In [ ]:
print(f"Filas: {df_spy.shape[0]}, Columns: {df_spy.shape[1]}")

In [ ]:
# output_name = 'spy_options_call_output_all_mc_price_gbm.csv'
output_name = 'spy_options_call_output_all_mc_price_lsm_2k.csv'
df_spy.to_csv(output_name, index=False)
print(f"se guardo el archivo CSV de todos los años")